In [63]:
from pymongo import MongoClient, ReturnDocument
from bson import ObjectId


MONGODB_URI = 'mongodb://localhost:27017/'
MONGO_DB_NAME = 'simple_api'


class MongoStorage(object):
        def __init__(self, collection_name):
            self.collection_name = collection_name

        def setup(self):
            self.client = MongoClient(MONGODB_URI)
            self.db = self.client[MONGO_DB_NAME]
            self.collection = self.db[self.collection_name]

        def finalize(self):
            self.client.close()

        def create(self, data):
            return self.collection.insert_one(data).inserted_id

        def update(self, data):
            filter_ = {'_id': ObjectId(data['_id'])}
            result = self.collection.find_one_and_replace(
                filter_, data, return_document=ReturnDocument.AFTER)

            return transform_id(result)

        def delete(self, id):
            result = self.collection.delete_one({'_id': ObjectId(id)})
            if result.deleted_count == 0:
                raise KeyError()

        def find(self):
            result = self.collection.find({})
            return [transform_id(doc) for doc in result]

        def read(self, id):
            result = self.collection.find_one({'_id': ObjectId(id)})
            if not result:
                raise KeyError()
            return transform_id(result)

        def bulk_insert(self, data):
            obj_ids = self.collection.insert_many(data)
            return [str(obj_id) for obj_id in obj_ids.inserted_ids]

        def clear(self):
            self.collection.delete_many({})
        
    
def transform_id(doc):
    doc['_id'] = str(doc['_id'])
    return doc

In [64]:
db = MongoStorage('test')

In [65]:
db.setup()

In [66]:
db.bulk_insert([{'txt': 'yo{}'.format(index)} for index in range(5)])

['5831cf849498f27cc6ad05e4',
 '5831cf849498f27cc6ad05e5',
 '5831cf849498f27cc6ad05e6',
 '5831cf849498f27cc6ad05e7',
 '5831cf849498f27cc6ad05e8']

In [9]:
list(cur)

[]

In [10]:
db.create({'txt': 'yo'})

ObjectId('5831bef09498f27cc6ad05cd')

In [11]:
oid = ObjectId('5831bef09498f27cc6ad05cd')

In [12]:
oid.binary

b'X1\xbe\xf0\x94\x98\xf2|\xc6\xad\x05\xcd'

In [13]:
str(oid)

'5831bef09498f27cc6ad05cd'

In [55]:
db.find()

[]

In [62]:
db.finalize()

In [54]:
db.clear()


In [67]:
db.delete('5831cf849498f27cc6ad05e4')

KeyError: '_id'

In [68]:
res = db.collection.delete_one({'_id': ObjectId('5831cf849498f27cc6ad05e4')})

In [69]:
res


In [70]:
res.raw_result

{'n': 0, 'ok': 1}

In [74]:
res = db.collection.delete_one({'_id': ObjectId('5831cf849498f27cc6ad05e7')})


In [75]:
res.deleted_count

1